In [2]:
import numpy as np
import pandas as pd

# Load Dataset

#### Train Data 가공

In [58]:
train = pd.read_csv("data/train.csv")
#train = pd.read_csv("C:\\Users\\keumkyu\\Desktop\\DSschool\\bike\\data\\train.csv")
print(train.shape)
train.head()


(10886, 12)


,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,3,10,13
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,0,1,1


In [59]:
#날짜 데이터로 변경하며 데이터 불러오기
train = pd.read_csv("data/train.csv", parse_dates=["datetime"])
#train = pd.read_csv("C:\\Users\\keumkyu\\Desktop\\DSschool\\bike\\data\\train.csv")
#train["datetime"].dt.year(month/day/hour/minute/second)
train["year"] = train["datetime"].dt.year
train["month"] = train["datetime"].dt.month
train["weekday"] = train["datetime"].dt.weekday
train["day"] = train["datetime"].dt.day
train["hour"] = train["datetime"].dt.hour

# dayofweek는 날짜에서 요일(월~일)을 가져오는 기능입니다.
# 값은 0(월), 1(화), 2(수), 3(목), 4(금), 5(토), 6(일) 을 나타냅니다.
train["datetime-dayofweek"] = train["datetime"].dt.dayofweek

train["datetime-year(str)"] = train["year"].astype('str')
train["datetime-month(str)"] = train["month"].astype('str')

# datetime-year(str)와 datetime-month(str) 문자열 두 개를 붙여서 datetime-year_month라는 새로운 컬럼을 추가합니다.
# 이 컬럼에는 2011-1부터 2012-12까지의 총 24의 경우의 수가 들어갑니다.
#train["datetime-year_month"] = train["datetime-year(str)"] + "-" + train["datetime-month(str)"]
train["datetime-year_month"] = train["datetime-year(str)"] + train["datetime-month(str)"]

#print(train.shape)
train.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,...,count,year,month,weekday,day,hour,datetime-dayofweek,datetime-year(str),datetime-month(str),datetime-year_month
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,...,16,2011,1,5,1,0,5,2011,1,20111
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,...,40,2011,1,5,1,1,5,2011,1,20111
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5,...,32,2011,1,5,1,2,5,2011,1,20111
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,3,...,13,2011,1,5,1,3,5,2011,1,20111
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,0,...,1,2011,1,5,1,4,5,2011,1,20111


In [60]:
train["season_1"] = train["season"] == 1
train["season_2"] = train["season"] == 2
train["season_3"] = train["season"] == 3
train["season_4"] = train["season"] == 4
train=train[train["humidity"]>0] #이거 넣으면, 점수 더 좋아짐

train.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,...,day,hour,datetime-dayofweek,datetime-year(str),datetime-month(str),datetime-year_month,season_1,season_2,season_3,season_4
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,...,1,0,5,2011,1,20111,True,False,False,False
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,...,1,1,5,2011,1,20111,True,False,False,False
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5,...,1,2,5,2011,1,20111,True,False,False,False
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,3,...,1,3,5,2011,1,20111,True,False,False,False
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,0,...,1,4,5,2011,1,20111,True,False,False,False


In [61]:
#windspeed 예측하기
from sklearn.ensemble import RandomForestRegressor

dataWind0 = train[train["windspeed"]==0]
dataWindNot0 = train[train["windspeed"]!=0]
rfModel_wind = RandomForestRegressor()
windColumns = ["season","weather","humidity","month","temp","year","atemp"]
rfModel_wind.fit(dataWindNot0[windColumns], dataWindNot0["windspeed"])

#dataWind0
wind0Values = rfModel_wind.predict(dataWind0[windColumns])
#wind0Values = rfModel_wind.predict(dataWind0[windColumns])
dataWind0["windspeedrev"] = wind0Values
dataWindNot0["windspeedrev"] = train["windspeed"]
dataWindNot0
#data = dataWindNot0.append(dataWind0)
#model.fit(X_train, y_train)
#predictions = model.predict(X_test)


c:\users\keumkyu\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
c:\users\keumkyu\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,...,hour,datetime-dayofweek,datetime-year(str),datetime-month(str),datetime-year_month,season_1,season_2,season_3,season_4,windspeedrev
5,2011-01-01 05:00:00,1,0,0,2,9.84,12.880,75,6.0032,0,...,5,5,2011,1,20111,True,False,False,False,6.0032
10,2011-01-01 10:00:00,1,0,0,1,15.58,19.695,76,16.9979,12,...,10,5,2011,1,20111,True,False,False,False,16.9979
11,2011-01-01 11:00:00,1,0,0,1,14.76,16.665,81,19.0012,26,...,11,5,2011,1,20111,True,False,False,False,19.0012
12,2011-01-01 12:00:00,1,0,0,1,17.22,21.210,77,19.0012,29,...,12,5,2011,1,20111,True,False,False,False,19.0012
13,2011-01-01 13:00:00,1,0,0,2,18.86,22.725,72,19.9995,47,...,13,5,2011,1,20111,True,False,False,False,19.9995
14,2011-01-01 14:00:00,1,0,0,2,18.86,22.725,72,19.0012,35,...,14,5,2011,1,20111,True,False,False,False,19.0012
15,2011-01-01 15:00:00,1,0,0,2,18.04,21.970,77,19.9995,40,...,15,5,2011,1,20111,True,False,False,False,19.9995
16,2011-01-01 16:00:00,1,0,0,2,17.22,21.210,82,19.9995,41,...,16,5,2011,1,20111,True,False,False,False,19.9995
17,2011-01-01 17:00:00,1,0,0,2,18.04,21.970,82,19.0012,15,...,17,5,2011,1,20111,True,False,False,False,19.0012
18,2011-01-01 18:00:00,1,0,0,3,17.22,21.210,88,16.9979,9,...,18,5,2011,1,20111,True,False,False,False,16.9979


In [62]:
#wind 0과 0 아닌애들을 합쳐주기
train=pd.concat([dataWind0, dataWindNot0]).sort_values('datetime')

#날씨 4 하나이므로 제거

train=train[train["humidity"] !=0]
train=train[train["count"] <900]

#### test data 가공

In [63]:
test = pd.read_csv("data/test.csv", parse_dates=["datetime"])

test["year"] = test["datetime"].dt.year
test["month"] = test["datetime"].dt.month
test["weekday"] = test["datetime"].dt.weekday
test["day"] = test["datetime"].dt.day
test["hour"] = test["datetime"].dt.hour

test["season_1"] = test["season"] == 1
test["season_2"] = test["season"] == 2
test["season_3"] = test["season"] == 3
test["season_4"] = test["season"] == 4

# dayofweek는 날짜에서 요일(월~일)을 가져오는 기능입니다.
# 값은 0(월), 1(화), 2(수), 3(목), 4(금), 5(토), 6(일) 을 나타냅니다.
test["datetime-dayofweek"] = test["datetime"].dt.dayofweek


test["datetime-year(str)"] = test["year"].astype('str')
test["datetime-month(str)"] = test["month"].astype('str')

# datetime-year(str)와 datetime-month(str) 문자열 두 개를 붙여서 datetime-year_month라는 새로운 컬럼을 추가합니다.
# 이 컬럼에는 2011-1부터 2012-12까지의 총 24의 경우의 수가 들어갑니다.
#test["datetime-year_month"] = test["datetime-year(str)"] + "-" + test["datetime-month(str)"]
test["datetime-year_month"] = test["datetime-year(str)"] + test["datetime-month(str)"]
print(test.shape)
test.head()


(6493, 22)


,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,year,...,day,hour,season_1,season_2,season_3,season_4,datetime-dayofweek,datetime-year(str),datetime-month(str),datetime-year_month
0,2011-01-20 00:00:00,1,0,1,1,10.66,11.365,56,26.0027,2011,...,20,0,True,False,False,False,3,2011,1,20111
1,2011-01-20 01:00:00,1,0,1,1,10.66,13.635,56,0.0000,2011,...,20,1,True,False,False,False,3,2011,1,20111
2,2011-01-20 02:00:00,1,0,1,1,10.66,13.635,56,0.0000,2011,...,20,2,True,False,False,False,3,2011,1,20111
3,2011-01-20 03:00:00,1,0,1,1,10.66,12.880,56,11.0014,2011,...,20,3,True,False,False,False,3,2011,1,20111
4,2011-01-20 04:00:00,1,0,1,1,10.66,12.880,56,11.0014,2011,...,20,4,True,False,False,False,3,2011,1,20111


In [64]:
#windspeed 예측하기
from sklearn.ensemble import RandomForestRegressor

testWind0 = test[test["windspeed"]==0]
testWindNot0 = test[test["windspeed"]!=0]
rfModel_wind = RandomForestRegressor()
windColumns = ["season","weather","humidity","month","temp","year","atemp"]
rfModel_wind.fit(testWindNot0[windColumns], testWindNot0["windspeed"])

#dataWind0
wind0Values = rfModel_wind.predict(testWind0[windColumns])
#wind0Values = rfModel_wind.predict(dataWind0[windColumns])
testWind0["windspeedrev"] = wind0Values
testWindNot0["windspeedrev"] = testWindNot0 ["windspeed"]
testWindNot0
#data = dataWindNot0.append(dataWind0)
#model.fit(X_train, y_train)
#predictions = model.predict(X_test)

c:\users\keumkyu\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
c:\users\keumkyu\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,year,...,hour,season_1,season_2,season_3,season_4,datetime-dayofweek,datetime-year(str),datetime-month(str),datetime-year_month,windspeedrev
0,2011-01-20 00:00:00,1,0,1,1,10.66,11.365,56,26.0027,2011,...,0,True,False,False,False,3,2011,1,20111,26.0027
3,2011-01-20 03:00:00,1,0,1,1,10.66,12.880,56,11.0014,2011,...,3,True,False,False,False,3,2011,1,20111,11.0014
4,2011-01-20 04:00:00,1,0,1,1,10.66,12.880,56,11.0014,2011,...,4,True,False,False,False,3,2011,1,20111,11.0014
5,2011-01-20 05:00:00,1,0,1,1,9.84,11.365,60,15.0013,2011,...,5,True,False,False,False,3,2011,1,20111,15.0013
6,2011-01-20 06:00:00,1,0,1,1,9.02,10.605,60,15.0013,2011,...,6,True,False,False,False,3,2011,1,20111,15.0013
7,2011-01-20 07:00:00,1,0,1,1,9.02,10.605,55,15.0013,2011,...,7,True,False,False,False,3,2011,1,20111,15.0013
8,2011-01-20 08:00:00,1,0,1,1,9.02,10.605,55,19.0012,2011,...,8,True,False,False,False,3,2011,1,20111,19.0012
9,2011-01-20 09:00:00,1,0,1,2,9.84,11.365,52,15.0013,2011,...,9,True,False,False,False,3,2011,1,20111,15.0013
10,2011-01-20 10:00:00,1,0,1,1,10.66,11.365,48,19.9995,2011,...,10,True,False,False,False,3,2011,1,20111,19.9995
11,2011-01-20 11:00:00,1,0,1,2,11.48,13.635,45,11.0014,2011,...,11,True,False,False,False,3,2011,1,20111,11.0014


In [65]:
#wind 0과 0 아닌애들을 합쳐주기
test=pd.concat([testWind0, testWindNot0]).sort_values('datetime')

## Train

In [66]:

#one hot encoding 한다고 무조건 좋은건 아니고, 관계성이 있는건 안하는게 더 좋다. 봄여름가을겨울은 봄에서 여름을 가면 더워지고, 가을에서 겨울가면 추워지는등의 관계성이 있으므로...
#타이타닉 embark는 이름이므로, 전혀 관계성이 없다
#feature_names=["year","hour","datetime-dayofweek","holiday","workingday","weather","atemp","humidity","windspeed","season_1","season_2","season3"]
feature_names=["year", "hour", "datetime-dayofweek","holiday","workingday","weather","atemp","temp","humidity","windspeedrev","season"]
feature_names

['year',
 'hour',
 'datetime-dayofweek',
 'holiday',
 'workingday',
 'weather',
 'atemp',
 'temp',
 'humidity',
 'windspeedrev',
 'season']

In [67]:
X_train = train[feature_names]
print(X_train.shape)
X_train.head()
#pd.pivot_table(train, index="humidity", values="count")

(10854, 11)


,year,hour,datetime-dayofweek,holiday,workingday,weather,atemp,temp,humidity,windspeedrev,season
0,2011,0,5,0,0,1,14.395,9.84,81,10.10092,1
1,2011,1,5,0,0,1,13.635,9.02,80,13.00135,1
2,2011,2,5,0,0,1,13.635,9.02,80,13.00135,1
3,2011,3,5,0,0,1,14.395,9.84,75,6.70201,1
4,2011,4,5,0,0,1,14.395,9.84,75,6.70201,1


In [68]:
X_test = test[feature_names]

print(X_test.shape)
X_test.head()

(6493, 11)


,year,hour,datetime-dayofweek,holiday,workingday,weather,atemp,temp,humidity,windspeedrev,season
0,2011,0,3,0,1,1,11.365,10.66,56,26.00270,1
1,2011,1,3,0,1,1,13.635,10.66,56,9.19843,1
2,2011,2,3,0,1,1,13.635,10.66,56,9.19843,1
3,2011,3,3,0,1,1,12.880,10.66,56,11.00140,1
4,2011,4,3,0,1,1,12.880,10.66,56,11.00140,1


#### 자전거 대여량인 count를 맞추는 것이기 때문에 값을 y로 넣어줌

In [69]:
label_name = "count"
y_train = train[label_name]
print(y_train.shape)
y_train.head()

(10854,)


0    16
1    40
2    32
3    13
4     1
Name: count, dtype: int64

## Use Decision Tree

In [70]:
from sklearn.tree import DecisionTreeRegressor
#랜덤하게 값을 계속 나누기 때문에, 그것을 방지하기 위하여 seed 값 설정
model = DecisionTreeRegressor(random_state = 37)
model

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=37, splitter='best')

## RandomForest

In [71]:
#데이터에서 일정 부분 샘플링해서 66%해서 넣고, 트리를 최대한으로 만들어서 그 트리끼리 투표해서 좋은것을 선택한다

In [72]:
from sklearn.ensemble import RandomForestRegressor
#랜덤하게 값을 계속 나누기 때문에, 그것을 방지하기 위하여 seed 값 설정
#model = RandomForestRegressor(random_state = 37)
model = RandomForestRegressor(random_state = 37)
model

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=37, verbose=0, warm_start=False)

## Evaluate

### Evaluate

머신러닝 모델을 학습시키기 전에, 측정 공식(Evaluation Metric)을 통해 학습한 모델의 성능이 얼마나 뛰어난지 정량적으로 측정해보겠습니다. 이번 [Bike Sharing Demand](https://www.kaggle.com/c/bike-sharing-demand) 경진대회에서 사용하는 측정 공식은 Root Mean Squared Logarithmic Error ([RMSLE](https://www.kaggle.com/c/bike-sharing-demand#evaluation)) 입니다.

$$ \sqrt{\frac{1}{n} \sum_{i=1}^n (\log(p_i + 1) - \log(a_i+1))^2 } $$

이 공식은 정답($a_i$, actual)과 예측값($p_i$, predict)의 차이가 크면 클수록 **페널티를 덜 주는** 방식으로 동작합니다. (```log(count + 1)```이 그 역할을 합니다) 이 점을 감안하여 파이썬과 [numpy](http://www.numpy.org/), [scikit-learn](http://scikit-learn.org/stable/)으로 **RMSLE** 공식을 구현해보겠습니다.

In [73]:
# numpy라는 패키지를 불러옵니다.
# 이 패키지는 선형대수(linear algebra) 패키지라고 불리는데,
# 현재는 간단하게 '수학 연산을 편하게 해주는 패키지'라고 이해하시면 됩니다.
import numpy as np

# scikit-learn 패키지의 metrics 모듈에서 make_scorer라는 함수를 가지고 옵니다.
# 이 함수는 파이썬을 구현한 측정 공식을 scikit-learn에서 사용할 수 있도록 변환해 줍니다.
from sklearn.metrics import make_scorer

# RMSLE 공식을 구현한 함수를 생성합니다.
# 이 함수는 예측값(predict)과 정답(actual)을 인자로 받습니다.
def rmsle(predict, actual):
    # predict와 actual을 numpy array로 변환합니다.
    # 이렇게 하면 수학 연산을 편하게 할 수 있습니다.
    predict = np.array(predict)
    actual = np.array(actual)
    
    # 공식에 쓰여진대로 predict와 actual에 log(count + 1)을 합니다.
    # 이 결과를 각각 log_predict와 log_actual에 대입합니다.
    log_predict = np.log(predict + 1)
    log_actual = np.log(actual + 1)
    
    # 공식에 쓰여진대로 log_predict와 log_actual을 빼서 차이를 구합니다.
    # 이 차이를 distance라는 이름의 새로운 변수에 할당합니다.
    distance = log_predict - log_actual
    
    # 공식에 쓰여진대로 distance를 제곱합니다.
    # 이 결과를 square_distance라는 이름의 새로운 변수에 할당합니다.
    square_distance = distance ** 2
    
    # 공식에 쓰여진대로 square_distance의 평균을 구합니다.
    # 이 결과를 mean_square_distance라는 이름의 새로운 변수에 할당합니다.
    mean_square_distance = square_distance.mean()
    
    # 공식에 쓰여진대로 mean_square_distance에 루트(sqrt)를 씌웁니다.
    # 이 결과를 score라는 이름의 새로운 변수에 할당합니다.
    score = np.sqrt(mean_square_distance)
    
    # score 변수를 반환합니다.
    return score

# scikit-learn의 make_scorer를 활용하여
# rmsle 함수를 scikit-learn의 다른 함수에서 사용할 수 있도록 변환합니다.
# 이 결과를 rmsle_score라는 이름의 새로운 변수에 할당합니다.
rmsle_score = make_scorer(rmsle)
rmsle_score

make_scorer(rmsle)

In [74]:
# cross validation 라이브러리 사용

from sklearn.model_selection import cross_val_score
#20개로 test set 만들기, 측정공식을 써서 테스트 셋이 어떤지 평가도 해줌
#바이크대회랑 비슷하게 평가해줄 수 있도록 평가지표를 변경
#1번 데이터 셋과 나머지 19개 비교, 2번 데이터와 나머지 20개 비교해서 예측값-정답의 평가 지표와 같이 비교해서 array로 내보내줌
scores = cross_val_score(model, X_train, y_train, cv =20, scoring = "neg_mean_squared_log_error")

scores = -1.0 * scores

score = scores.mean()

#포멧설정 소수점 5개 까지

Score = format(score, '.5f')
Score

'0.15263'

In [75]:
# scikit-learn의 model_selection 모듈에서 cross_val_score 함수를 가져옵니다.
# 이 함수는 Cross Validation의 모든 기능을 한 번에 실행합니다.
from sklearn.model_selection import cross_val_score

# cross_val_score를 실행합니다. 실행할 때는 다음의 옵션이 들어갑니다.
# 1) model. 점수를 측정할 머신러닝 모델(현재는 Random Forest)이 들어갑니다.
# 2) X_train. train 데이터의 feature 입니다.
# 3) y_train. train 데이터의 label 입니다.
# 4) cv. Cross Validation에서 데이터를 조각낼(split) 갯수입니다. 총 20조각을 내야하기 때문에 20을 대입합니다.
# 5) scoring. 점수를 측정할 공식입니다. 앞서 구현한 RMSLE를 적용합니다.
# 마지막으로, 이 함수의 실행 결과의 평균(mean)을 구한 뒤 score라는 이름의 새로운 변수에 할당합니다.
score = cross_val_score(model, X_train, y_train,
                        cv=20, scoring=rmsle_score).mean()

# score 변수에 할당한 점수를 출력합니다.
print("Score = {0:.5f}".format(score))

Score = 0.38196


In [76]:
model.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=37, verbose=0, warm_start=False)

In [77]:
predictions = model.predict(X_test)
test.head()
#print(predictions.shape)
#predictions

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,year,...,hour,season_1,season_2,season_3,season_4,datetime-dayofweek,datetime-year(str),datetime-month(str),datetime-year_month,windspeedrev
0,2011-01-20 00:00:00,1,0,1,1,10.66,11.365,56,26.0027,2011,...,0,True,False,False,False,3,2011,1,20111,26.00270
1,2011-01-20 01:00:00,1,0,1,1,10.66,13.635,56,0.0000,2011,...,1,True,False,False,False,3,2011,1,20111,9.19843
2,2011-01-20 02:00:00,1,0,1,1,10.66,13.635,56,0.0000,2011,...,2,True,False,False,False,3,2011,1,20111,9.19843
3,2011-01-20 03:00:00,1,0,1,1,10.66,12.880,56,11.0014,2011,...,3,True,False,False,False,3,2011,1,20111,11.00140
4,2011-01-20 04:00:00,1,0,1,1,10.66,12.880,56,11.0014,2011,...,4,True,False,False,False,3,2011,1,20111,11.00140


## submit

In [78]:
submission = pd.read_csv("data/sampleSubmission.csv")
submission["count"] = predictions
#X_test.head()
print(submission.shape)
submission.head()

(6493, 2)


,datetime,count
0,2011-01-20 00:00:00,11.4
1,2011-01-20 01:00:00,4.4
2,2011-01-20 02:00:00,4.5
3,2011-01-20 03:00:00,3.4
4,2011-01-20 04:00:00,3.1


In [79]:
#인덱스를 타이타닉의 passenger처럼 될만한게 없어서 따로 안줬떠니, 인덱스까지 저장되므로, 인덱스를 없애고 저장한다
submission.to_csv("data/baseline-script9-huminity.csv", index=False)